In [1]:
using Revise
include("ngrams.jl")
include("datasetloader.jl")
LANGS = ["ara", "bel", "ben", "bul", "cat", "ces", "dan", "deu", "ell", 
"eng", "epo", "fas", "fin", "fra", "hau", "hbs", "heb", "hin", "hun", 
"ido", "ina", "isl", "ita", "jpn", "kab", "kor", "kur", "lat", "lit", 
"mar", "mkd", "msa", "nds", "nld", "nor", "pol", "por", "ron", "rus", 
"slk", "spa", "swa", "swe", "tat", "tgl", "tur", "ukr", "vie", "yid", 
"zho"]
G = [lang => load_ngrams(joinpath("ngrams", lang * ".txt")) for lang in LANGS]

50-element Vector{Pair{String, Dict{Tuple{UInt8, Vararg{UInt8}}, Float64}}}:
 "ara" => Dict((0xa4, 0x20) => 4.30232865576655e-5, (0xa3,) => 0.07035901432661235, (0x63, 0x61) => 0.00029467622074500394, (0x83,) => 0.07762947735596568, (0x68, 0x65) => 0.00010555566116238947, (0x63, 0x20, 0x6d) => 0.00010555566116238947, (0x79, 0x20, 0x6d) => 0.00016273164429201712, (0x6e, 0x73, 0x20) => 8.79630509686579e-5, (0x20, 0x72, 0x65, 0x66) => 0.00010555566116238947, (0x69, 0x6e, 0x20, 0x6d) => 0.00013194457645298685…)
 "bel" => Dict((0xd0, 0xb0, 0x20, 0xd0) => 0.05356847819684612, (0xd0, 0xbd, 0xd1, 0x83) => 0.005204917113475131, (0x63, 0x61) => 9.886008612132829e-5, (0xd1, 0x87) => 0.03788978898882851, (0xd0, 0xb3, 0xd0, 0xbb) => 0.0028589731977762345, (0x68, 0x65) => 0.0002951312235264679, (0xd0, 0xb2, 0xd0, 0xbe) => 0.007195015202415148, (0xb3, 0x20) => 0.0017472407209811849, (0xbb, 0xd0, 0xb5, 0xd0) => 0.007499456420347099, (0xb5, 0xd0, 0xbc, 0xd0) => 0.0009426666745662483…)
 "ben" => Dict((0

In [2]:
sort([l=>sum(values(D)) for (l,D) in G], by=last, rev=true)

50-element Vector{Pair{String, Float64}}:
 "rus" => 26.73726375666268
 "mkd" => 26.151445910151473
 "bul" => 24.676605841761564
 "ukr" => 23.696178807599757
 "hbs" => 22.62492470321158
 "bel" => 20.56416193833829
 "tat" => 18.67189188702598
 "ita" => 18.426543862944676
 "spa" => 18.096998582168354
 "epo" => 17.57372192244551
       ⋮
 "vie" => 8.955603522438562
 "kur" => 8.641355090406647
 "ben" => 8.324424528866977
 "heb" => 7.528650473241067
 "yid" => 6.961059662312389
 "zho" => 6.235235105542326
 "ell" => 5.538190814675658
 "jpn" => 5.157479514067984
 "kor" => 4.648805289067819

In [3]:
function norm_table!(t)
    D = last(t)
    vs = sum(values(D))
    for (k, v) in D
        D[k] /= vs
        D[k] = log(D[k])
    end
end

norm_table! (generic function with 1 method)

In [4]:
norm_table!.(G);

In [5]:
LOG_T = last.(G)

50-element Vector{Dict{Tuple{UInt8, Vararg{UInt8}}, Float64}}:
 Dict((0xa4, 0x20) => -12.362077095707182, (0xa3,) => -4.962452422918042, (0x63, 0x61) => -10.437941415882628, (0x83,) => -4.864116115793013, (0x68, 0x65) => -11.46458020492565, (0x63, 0x20, 0x6d) => -11.46458020492565, (0x79, 0x20, 0x6d) => -11.031716122629371, (0x6e, 0x73, 0x20) => -11.646901761719604, (0x20, 0x72, 0x65, 0x66) => -11.46458020492565, (0x69, 0x6e, 0x20, 0x6d) => -11.241436653611439…)
 Dict((0xd0, 0xb0, 0x20, 0xd0) => -5.950344326251024, (0xd0, 0xbd, 0xd1, 0x83) => -8.281701350454895, (0x63, 0x61) => -12.245354827950015, (0xd1, 0x87) => -6.296623472022127, (0xd0, 0xb3, 0xd0, 0xbb) => -8.88084258937918, (0x68, 0x65) => -11.151640324025045, (0xd0, 0xb2, 0xd0, 0xbe) => -7.957916674739691, (0xb3, 0x20) => -9.373267315271399, (0xbb, 0xd0, 0xb5, 0xd0) => -7.916474587362034, (0xb5, 0xd0, 0xbc, 0xd0) => -9.990347660244385…)
 Dict((0xa3,) => -6.5289873003078105, (0x63, 0x61) => -11.927710358455563, (0x68, 0x65) => -1

In [6]:
DEFAULT_LOGP = (minimum.(values.(last.((G))))|>minimum)

-16.778835231252533

In [52]:
TV = TatoebaDataset("corpus/tatoeba", "tatoeba_test.txt", langs=LANGS)
TV[rand(1:length(TV))]

"¿Quieres que me quede?" => "spa"

In [7]:
function likelihood(t, logt, default_logp=DEFAULT_LOGP)
    sc = 0.
    for (code, p) in t
        sc += p * get(logt, code, default_logp)
    end
    sc
end
    

likelihood (generic function with 2 methods)

In [88]:
x, y = TV[rand(1:length(TV))]

"A águia voa pelo vale ensolarado." => "por"

In [8]:
function detector_lh(text; ngram=5, languages=LANGS, logtable=LOG_T)
    t = merged_ngrams(text, ngram)
    lhs = likelihood.(Ref(t), logtable)
    languages[argmax(lhs)]
end


detector_lh (generic function with 1 method)

In [9]:
include("benchmark.jl")

showtable (generic function with 2 methods)

In [10]:
WV = WikiDataSet("corpus/wikipedia/test", langs=LANGS)
TV = TatoebaDataset("corpus/tatoeba", "tatoeba_test.txt", langs=LANGS)

521469-element TatoebaDataset:
             "Pardonu, permesu al mi indiki tri erarojn en la supra artikolo." => "epo"
                                          "Mi veturigos vin al la flughaveno." => "epo"
                                                                 "Kio mankas?" => "epo"
                                            "Aĥ, kiam ili renkontiĝos denove?" => "epo"
 "Kiam mi estis infano, mi pasigis multajn horojn legante sola en mia ĉambro." => "epo"
                                              "Ŝajnas, ke la trajno malfruas." => "epo"
                                                                "Venu rapide!" => "epo"
                                                   "Li montris al mi albumon." => "epo"
                                                             "Estis du kukoj." => "epo"
                                      "Mia filo scipovas jam nombri ĝis cent." => "epo"
                                                                               ⋮
        

In [11]:
WB = benchmark(
    # "likelihood-n5"=>detector_lh, 
    "likelihood-n4"=>t->detector_lh(t, ngram=4),
    "likelihood-n3"=>t->detector_lh(t, ngram=3),
dataset=WV, languages=LANGS)

likelihood-n4

131.529808 seconds (1.63 G allocations: 28.839 GiB, 3.52% gc time, 0.53% compilation time)
likelihood-n3

 96.410937 seconds (885.16 M allocations: 16.351 GiB, 2.66% gc time, 0.07% compilation time)


2-element Vector{Any}:
 "likelihood-n4" => [1.0, 0.99, 1.0, 0.945, 0.995, 0.98, 0.99, 0.96, 0.995, 1.0  …  1.0, 0.985, 0.98, 0.995, 0.985, 0.98, 1.0, 0.95, 0.97, 0.995]
 "likelihood-n3" => [1.0, 0.99, 1.0, 0.89, 0.995, 0.98, 0.98, 0.96, 1.0, 1.0  …  1.0, 0.99, 0.98, 0.995, 0.985, 0.985, 0.995, 0.96, 0.97, 0.995]

In [13]:
showtable(WB, LANGS, threshold=-0.5)

|               | Average | ara     | bel    | ben     | bul    | cat    | ces    | dan    | deu    | ell     | eng     | epo     | fas    | fin    | fra     | hau    | hbs   | heb     | hin    | hun     | ido    | ina    | isl    | ita    | jpn    | kab    | kor    | kur    | lat    | lit     | mar    | mkd    | msa    | nds    | nld    | nor    | pol     | por    | ron    | rus    | slk    | spa     | swa    | swe    | tat    | tgl    | tur    | ukr     | vie    | yid    | zho    |
|---------------|---------|---------|--------|---------|--------|--------|--------|--------|--------|---------|---------|---------|--------|--------|---------|--------|-------|---------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|--------|
| likelihood-n4 |  95.

|               | Average | ara     | bel    | ben     | bul    | cat    | ces    | dan    | deu    | ell     | eng     | epo     | fas    | fin    | fra     | hau    | hbs   | heb     | hin    | hun     | ido    | ina    | isl    | ita    | jpn    | kab    | kor    | kur    | lat    | lit     | mar    | mkd    | msa    | nds    | nld    | nor    | pol     | por    | ron    | rus    | slk    | spa     | swa    | swe    | tat    | tgl    | tur    | ukr     | vie    | yid    | zho    |
|---------------|---------|---------|--------|---------|--------|--------|--------|--------|--------|---------|---------|---------|--------|--------|---------|--------|-------|---------|--------|---------|--------|--------|--------|--------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|--------|---------|--------|--------|--------|--------|---------|--------|--------|--------|--------|--------|---------|--------|--------|--------|
| likelihood-n4 |  95.53% | 100.00% | 99.00% | 100.00% | 94.50% | 99.50% | 98.00% | 99.00% | 96.00% |  99.50% | 100.00% | 100.00% | 98.50% | 99.00% | 100.00% | 98.50% | 4.00% | 100.00% | 90.50% |  99.50% | 89.50% | 93.00% | 98.50% | 99.00% | 90.00% | 98.50% | 93.50% | 99.00% | 97.00% | 100.00% | 92.00% | 98.00% | 99.00% | 97.50% | 97.00% | 92.50% |  98.50% | 99.50% | 91.50% | 99.50% | 94.00% | 100.00% | 98.50% | 98.00% | 99.50% | 98.50% | 98.00% | 100.00% | 95.00% | 97.00% | 99.50% |
| likelihood-n3 |  95.30% | 100.00% | 99.00% | 100.00% | 89.00% | 99.50% | 98.00% | 98.00% | 96.00% | 100.00% | 100.00% | 100.00% | 99.00% | 99.00% | 100.00% | 98.50% | 5.00% | 100.00% | 90.50% | 100.00% | 88.00% | 85.50% | 98.50% | 98.00% | 90.00% | 99.00% | 93.50% | 99.00% | 96.50% | 100.00% | 92.00% | 97.00% | 99.00% | 98.50% | 97.00% | 92.50% | 100.00% | 99.00% | 89.50% | 99.50% | 96.00% | 100.00% | 99.00% | 98.00% | 99.50% | 98.50% | 98.50% |  99.50% | 96.00% | 97.00% | 99.50% |


In [13]:
hbs = WikiDataSet("corpus/wikipedia/test", langs=["hbs"]);

In [14]:
x,y = hbs[rand(1:length(hbs))]
detector_lh(x, ngram=4), y

("mkd", "hbs")

In [14]:
WB2 = benchmark(
    "likelihood-n5"=>detector_lh, 
    # "likelihood-n4"=>t->detector_lh(t, ngram=4),
    # "likelihood-n3"=>t->detector_lh(t, ngram=3),
dataset=WV, languages=("mkd", "hbs"))

likelihood-n5  7.939650 seconds (90.26 M allocations: 1.665 GiB, 4.16% gc time, 1.21% compilation time)


1-element Vector{Any}:
 "likelihood-n5" => [0.98, 0.04]